In [1]:
###############################
# Import the necessary modules
###############################

# The PyBullet physics simulation library
import pybullet as pb
import pybullet_data

# Numpy for numerical calculations and manipulations
import numpy as np
import math

# Matplotlib to create the necessary plots
import matplotlib.pyplot as plt

In [2]:
##############################################################
# Create an instance of the Physics Server and connect to it
##############################################################

# Use p.DIRECT to connect to the server without rendering a GUI
# Use p.GUI to create a GUI to render the simulation
client = pb.connect(pb.GUI) # or p.GUI


# Load the URDF of the plane that forms the ground
pb.setAdditionalSearchPath(pybullet_data.getDataPath()) # Set the search path to find the plane.urdf file
plane = pb.loadURDF("plane.urdf")


# Load the URDF of the robot
scara = pb.loadURDF("scara_robot_left.urdf")

In [3]:
##################################################
# Set the necessary parameters for the simulation
##################################################

# Set the Gravity vector
pb.setGravity(0,0,-9.81, physicsClientId = client)

# Set the simulation time-step
pb.setTimeStep(0.001) #The lower this is, more accurate the simulation 

# You can be faster than real-time if you choose
#p.setRealTimeSimulation(0)  # we want to be faster than real time.

In [4]:
#################################
# Enable the motors on the joints 
#################################

# This step is required to enable torque control. Refer to the documentation for more details.
pb.setJointMotorControl2(scara, 1, pb.VELOCITY_CONTROL,targetVelocity=0.5, force=5)
#pb.setJointMotorControl2(scara, 2, pb.VELOCITY_CONTROL, force=0)

In [5]:
dt     = 0.00001 # Simulation time-step
f      = 1.0 # Frequency of oscillation (1 Hz)
omega  = 2*math.pi*f # Angular frequency
theta0 = 0 # Start position
p_des = np.zeros(100000)
for i in range(100000):
    t = i*dt
    p_des[i] = 10*t

In [6]:
##########################################
# Control loop to follow the trajectory
##########################################
# Kinematics for serial-2
    
    
dt     = 0.00001 # simulation time-step
p_gain = 1000 # Proportional gain
d_gain = 500 # Derivative gain
error  = 0
error_old = 0

for i in range(100000):
    pos1, _, _, _ = pb.getJointState(scara,1)
    
    error_old = error
    error = p_des[i] - pos1
    error_d = (error - error_old)/dt
    control_force = p_gain * error + d_gain * error_d
    control_force = np.clip(control_force, -500, 500)
    pb.setJointMotorControl2(scara, 1, pb.TORQUE_CONTROL, force=control_force)
    pb.stepSimulation()


# Check if the robot has reached the desired position
pos1, _, _, _ = pb.getJointState(scara, 1)
pos2, _, _, _ = pb.getJointState(scara, 2)

In [7]:
pb.disconnect()